# Solutions

---
### 2.3.1 Exercise:

Add the following data to the `mySerie1` pandas Series object as a new row:

```
GeneK   25
```

In [ ]:
# Make a pandas Series from a dictionary and append it to the `mySerie1` pandas Series object. 
mySerie2 = pd.Series({"GeneK" : 25})
mySerie1 = mySerie1.append(mySerie2)

In [ ]:
# Alternatively and a bit shorter
mySerie1['GeneK'] = 25

---
### Exercise 2.4.1
- Select the number of counts in *GeneD* for the second and third experiment. 
- Add a new column to the dataframe with the average of the three experiments.

In [ ]:
# Subquestion one
#df[['counts_exp2','counts_exp3']][3:4]
#df.iloc[[3],[1,2]]
#df.loc[['GeneD'],['counts_exp2','counts_exp3']]

# Subquestion two
#df['avg'] = df.sum(axis = 1) / len(df.columns)
#df['avg'] = (df['counts_exp1'] + df['counts_exp2'] + df['counts_exp3']) / 3

---
### Exercise 2.4.2

- Search in the pandas documentation for the median method and add a column that describes the median countvalues per gene.
- Search in the pandas documentation for a method that will count all of the values of one experiment and add it as an extra row to the table. 
- Remove the row with the sum of the counts that we added in the previous step. 

In [ ]:
# Subquestion one
df.median(axis = 1, skipna = True)
df[['counts_exp1','counts_exp2','counts_exp3']].median(axis = 1, skipna = True)
df['median'] = df[['counts_exp1','counts_exp2','counts_exp3']].median(axis = 1, skipna = True)

# Subquestion two 
df.sum()
df[['counts_exp1','counts_exp2','counts_exp3']].sum()
df.loc['sum'] = df[['counts_exp1','counts_exp2','counts_exp3','avg']].sum()
df

# Subquestion three
df.drop('sum', axis = 0, inplace=True)

---
### 2.5.1 Exercise
Search for the parameters of `.read_csv` that you need in order to read in the `metagenic.csv` file where:
- chromosomes are the index of the rows, and 
- only the first 10 rows are imported.  

In [ ]:
metagenic_sub = pd.read_csv('data/metagenic.csv', index_col = 'chr', nrows = 10)
metagenic_sub

---
### 2.5.2 Exercise
Can you find a method that will retrieve the indices of all the virginica flowers? 

In [ ]:
iris.loc[iris['species'] == 'virginica'].index

---
### 2.5.3 Exercise
From the file `metagenic.csv`:
1. Sort the table based on the counts in exons in descending way  
2. Make a subselection of chromosomes with at least 15 counts in introns. 

In [ ]:
metagenic = pd.read_csv('data/metagenic.csv', index_col = 'chr')
#metagenic

In [ ]:
# Subquestion one
metagenic.sort_values(by=['exon'], ascending = False)

# Subquestion two
metagenic[metagenic['intron'] >= 15]

---
### 2.5.4 Exercise
In this exercise, derived from the [GTN](https://galaxyproject.github.io/training-material/topics/transcriptomics/tutorials/rna-seq-viz-with-heatmap2/tutorial.html), we will prepare the data to create a heatmap (*Exercise 3.x.y?*) of the top differentially expressed genes in an RNA-seq counts dataset. 
- [`counts`](https://zenodo.org/record/2529926/files/limma-voom_normalised_counts)
- [`de_genes`](https://zenodo.org/record/2529926/files/limma-voom_luminalpregnant-luminallactate)  

The latter file contains the results from comparing gene expression in the luminal cells in the pregnant versus lactating mice. It includes genes that are not significantly differentially expressed. We’ll call genes significantly differentially expressed in this dataset if they pass the thresholds of `adjusted P-value < 0.01` and `fold change of > 1.5 (log2FC of 0.58)`. Filter the top 20 DE genes from that table and create a joint dataframe that contains only the following columns and looks like this:

| SYMBOL_x |  MCL1.DG |  MCL1.DH |  MCL1.DI |  MCL1.DJ |   MCL1.DK |   MCL1.DL |  MCL1.LA |  MCL1.LB |  MCL1.LC |  MCL1.LD |  MCL1.LE |  MCL1.LF |
|---------:|---------:|---------:|---------:|---------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|----------|
|     Ggt1 | 6.732347 | 6.556047 | 6.558849 | 6.586562 |  6.437596 |  6.394067 | 5.193118 | 5.526432 | 4.223990 | 4.341605 | 7.243899 | 7.354535 |
|  Slc39a4 | 2.722153 | 3.027691 | 2.175532 | 1.993214 | -0.193255 | -0.016902 | 3.071502 | 2.928202 | 6.472918 | 6.526836 | 2.430346 | 1.847241 |
|      Ppl | 5.102274 | 4.900942 | 5.755087 | 5.951023 |  6.851420 |  6.881858 | 7.359977 | 7.732010 | 8.227118 | 8.437499 | 4.646145 | 4.798986 |
| ...   |     ...     |      ...    |   ...       |        ...  |    ...       |         ...  |       ...  |        ...   |      ...    |      ...    |       ...   |       ...   |

Save the file as a csv-file in the data-folder. 

In [ ]:
import pandas as pd
counts = pd.read_csv('https://zenodo.org/record/2529926/files/limma-voom_normalised_counts', sep='\t')
de_genes = pd.read_csv('https://zenodo.org/record/2529926/files/limma-voom_luminalpregnant-luminallactate', sep='\t')

In [ ]:
# Filters
p_adj_lim = 0.01
logFC_lim = 0.58

# Filter the non-significantly differentially expressed genes out
de_genes = de_genes[abs(de_genes['logFC']) > logFC_lim] 
de_genes = de_genes[de_genes['P.Value'] < p_adj_lim] 

# Sort the remaining significantly expressed genes (highest DE genes on top)
de_genes = de_genes.sort_values('P.Value')

# Filter the top 20 DE genes
de_top20 = de_genes.iloc[0:20, :]

# Create dataframe for heatmap that is a joined dataframe of the two imported data files
df_heatmap = pd.merge(counts, de_top20, on='ENTREZID')

# Make a subselection of the columns (genes and DE, see the df above)
df_heatmap = df_heatmap.iloc[:, [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]

# Set the names of the genes as the row index
df_heatmap = df_heatmap.set_index('SYMBOL_x')

# Store the dataframe in a csv file for later usage. 
df_heatmap.to_csv('data/heatmap_data.csv')

df_heatmap

---
### 3.2.3.1 Exercise:
Plot the same barplot but only for Belgium vs the Netherlands. Find a barplot argument that selects which country is selected and hence plotted (instead of making another subselection of the pandas dataframe). 

---
### 3.2.5.1 Exercise
Plot the same graph as given here above with the following modifications:
- Use a white background
- Color the dots according to its strand orientations
- Change x- and y-labels
- Remove the upper and right spine of the plots (http://seaborn.pydata.org/generated/seaborn.despine.html)

In [ ]:
# The following line will make a grid on a white background
sns.set(style="white")

# Plot the figure
ax = sns.scatterplot(x='log2(FC)', y='Log10 P-value', data=volc, hue='Strand')

# Change label names
plt.xlabel('log2(Fold Change)')
plt.ylabel('-log10(P-Value)')

# Despine plot
sns.despine()

#plt.savefig('img/volcanoplot.png')

---
### 3.3.1 Exercise 
Edit the lineplot above with the following adjustments:
- Set the style to a white background with ticks on the axes
- Set the context to a paper format
- Change the figure size
- Rename the axes and title of the plot 

Use the information here below to change the lay-out of the plot.

In [ ]:
# 1. Import data

# 2. Set style of the plot
sns.set_style("ticks")
sns.set_context("paper")

# 3. Define/create the plot
plt.figure(figsize=(10,4))
ax = sns.lineplot(x = 'Date', y = 'Cost per Mb', data = seqcost)

# 4. Tweak lay-out
ax.set(xlabel='Years', ylabel='Cost per Mb', title='Cost of sequencing')
plt.setp(axs, yticks=[])
plt.setp(ax.get_xticklabels(), rotation=90)
sns.despine()
#plt.savefig('img/seqcost.png')

---
### 3.3.2 Extra exercise
Make two subplots underneath each other that plot the Cost per Mb over years and the Total cost. 
Find more information on subplots [here](https://matplotlib.org/3.1.0/gallery/subplots_axes_and_figures/subplots_demo.html).

In [ ]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize= (10,7))
sns.despine()
fig.suptitle('Vertically stacked subplots')
sns.lineplot(x = 'Date', y = 'Cost per Mb', data = seqcost, color = 'b', ax = axs[0])
sns.lineplot(x = 'Date', y = 'Cost per Genome', data = seqcost, color = 'r', ax = axs[1])

#plt.setp(axs, yticks=[])
plt.setp(axs[0].get_xticklabels(), rotation=90)
plt.setp(axs[1].get_xticklabels(), rotation=90)
plt.tight_layout()
#plt.savefig('img/subplots_seqcost.png')

---
### 5.2.1 Exercise
Calculate the GC-content in the following sequence:
```
GATTACCACTCACTGACTCACTGACACGAGACCTATACATGATCGCCGGATGATACGAGAATTACTGACGACTAATCCCGGATACTGCATACACTGACGACGACT
```
- Use the `.count()` method as shown above
- Search through Bio.SeqUtils for a function that might help you

In [ ]:
ex_seq = Seq("GATTACCACTCACTGACTCACTGACACGAGACCTATACATGATCGCCGGATGATACGAGAATTACTGACGACTAATCCCGGATACTGCATACACTGACGACGACT")

In [ ]:
# GC content
100*float(ex_seq.count("G")+ex_seq.count("C"))/len(ex_seq)

In [ ]:
# Use the built-in method of Bio.SeqUtils
from Bio.SeqUtils import GC
GC(ex_seq)

# the Bio.SeqUtils.GC() function should automatically cope with mixed case 
# sequences and the ambiguous nucleotide S which means G or C.


--- 
### 5.3.1 Exercise
Can you concatenate the following sequences using a for-loop?
- Seq("ACGT")
- Seq("GCTA")
- Seq("TACG")

In [ ]:
# Method 1
seq1 = Seq("ACGT")
seq2 = Seq("GCTA")
seq3 = Seq("TACG")
list_of_seqs = [seq1, seq2, seq3]

concatenated = Seq('')

for each_seq in list_of_seqs:
    concatenated += each_seq
    print(concatenated)
    
concatenated

In [ ]:
# Method 2
list_of_seqs = [Seq("ACGT"), Seq("GCTA"),Seq("TACG")]
sum(list_of_seqs, Seq(""))

---
## 5.7 Exercise

Identifying genes is possible by looking for open reading frames (ORFs). For eukaryotic genes we know that there is a complex interaction between promotors, start codons, exons and introns. Nonetheless, for prokaryotic and virus genes this approach would still be useful. 

Depending on the organism you also need to use the according codon table. In this example we're using a bacterial plasmid fasta file for which we need to use codon [table 11](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi#SG11). By using the following block of code, we will store the sequence in the variable `record`, define the tranlate tables and define that a possible protein needs to be of a minimum length of 100 AA's. 

In [ ]:
from Bio import SeqIO
record = SeqIO.read("data/NC_005816.fna", "fasta")
table = 11
min_pro_len = 100

The output might look something like this: 
```
GCLMKKSSIVATIITILSGSANAASSQLIP...YRF, - length 315, strand 1, frame 0
KSGELRQTPPASSTLHLRLILQRSGVMMEL...NPE, - length 285, strand 1, frame 1
NQIQGVICSPDSGEFMVTFETVMEIKILHK...GVA, - length 355, strand 1, frame 2
QGSGYAFPHASILSGIAMSHFYFLVLHAVK...CSD, - length 114, strand -1, frame 0
```

In [ ]:
# Make a for loop to access first the + strand and then the - strand
for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
    
    # Allow frame shifts 
    for frame in range(3):
        
        # Length of sequence adjusted for frame shift 
        length = 3 * ((len(record)-frame)  // 3)
        
        # Translate sequence (nuc) to AA's. Split at * which decodes for a stop codon. These are all the ORFs
        for pro in nuc[frame:frame+length].translate(table).split("*"):
            
            # if length of ORF is >= predefined length
            if len(pro) >= min_pro_len:
                
                # Print results
                print(f"{pro[:30]}...{pro[-3:]}, - length {len(pro)}, strand {strand}, frame {frame}") 

In [ ]:
# Alternatively, function-wise. Define the arguments and function
def extract_ORF(record, strand, frame, table, min_len):
    """extract_ORF accepts a sequence record object as argument together with a strand orientation 
    and frameshift and will give you as an output all of the possible ORFs from that sequence record object
    that are longer than a predefined minimal length of AAs using a specific codon table"""
    
    # Create empty dataframe that will store all the information
    seq_info = pd.DataFrame(columns=["Sequence", "Length", "Strand", "Frame"])

    if strand == -1:
        print("Antisense strand orientation")
        record_seq = record.seq.reverse_complement()
    elif strand == +1:
        print("Sense strand orientation")
        record_seq = record.seq
    else:
        print("No correct strand information")
    
    length = 3 * ((len(record)-frame)  // 3)  
    if frame == 0:
        print("No frameshift")
    elif frame == 1:
        print("Frameshift 1")
        record_seq =  record_seq[frame:frame+length]
    elif frame == 2: 
        record_seq =  record_seq[frame:frame+length]
        print("Frameshift 2")

    for sub_prot in record_seq.translate(table=table).split('*'):
        if len(sub_prot) >= min_len:
            seq_info = seq_info.append({'Sequence': str(sub_prot), 
                                        'Length': len(sub_prot),
                                        'Strand': strand,
                                        'Frame': frame}, ignore_index = True)
    return seq_info

extract_ORF(record=record, strand=+1, frame=2, table=11, min_len=100)

---
### 6.1.1 Exercise
Find the title of all the articles related to the genbank entry 'NC_005816'. Import this file using the following block of code.  

Extra: Create a list of URL-links that brings you directly to the article. For this you can use the Pubmed ID in combination with `https://pubmed.ncbi.nlm.nih.gov/`. 


Hint: look at the section of *references* of [this link](https://biopython.readthedocs.io/en/latest/chapter_seq_annot.html)

In [ ]:
from Bio import SeqIO
record = SeqIO.read("data/NC_005816.gb","gb")

In [ ]:
# First create an empty list that will store the URL-links. 
list_pubmed = []
base_url = 'https://pubmed.ncbi.nlm.nih.gov/'
for ref in record.annotations['references']:
    # The titles of the journals are written in the title submodule
    print(ref.title)
    
    # Extra: if there is a pubmed ID
    if not ref.pubmed_id:
        # Prepare to add pubmed ID together with Base URL
        url_link = (base_url, ref.pubmed_id)
        # Join the pubmed ID with Base URL and append this to the list of Pubmed ID's
        list_pubmed.append(''.join(url_link))
print(list_pubmed)

--- 
### 7.1.1 Exercise
Make a list that contains the organism of each record in the `data/ls_orchid.gbk`-file. 

Tip: you should make an empty list, iterate over all the records, access the organism and append it to the  list. 

In [ ]:
from Bio import SeqIO

In [ ]:
# Method 1 by using the annotations (cleaner)
all_species= []
for seq_record in SeqIO.parse("data/ls_orchid.gbk","genbank"):
    all_species.append(seq_record.annotations["organism"])
    
print(set(all_species))

In [ ]:
# Method 2 by using the description (can be a bit tricky if the name of the organism is not on the second location)
all_species = []
for seq_record in SeqIO.parse("data/ls_orchid.fasta","fasta"):
    all_species.append(seq_record.description.split()[1])
print(all_species)

## 8.3 Exercise
Write a script that blasts the top 5 overrepresented sequences in a fastq-file. Save the following information in a pandas dataframe: title, e-value and score. 


Here is a table that is part of the output of a FastQC process. The raw data can be obtained from the zipped folder that is always created as part of the process. This part represents the overrepresented sequences in a fastq file. The file that contains the data is stored under `data/overrepresented_sequences.txt`. 


```
#Sequence	Count	Percentage	Possible Source
GCGCCAGGTTCCACACGAACGTGCGTTCAACGTGACGGGCGAGAGGGCGG	634749	0.9399698125201895	No Hit
GCCAGGTTCCACACGAACGTGCGTTCAACGTGACGGGCGAGAGGGCGGCC	437871	0.6484224816077345	No Hit
GGGGACAGTCCGCCCCGCCCCCCACCGGGCCCCGAGAGAGGCGACGGAGG	319343	0.47289996493044484	No Hit
GGCTTCCTCGGCCCCGGGATTCGGCGAAAGCTGCGGCCGGAGGGCTGTAA	310651	0.4600283926862577	No Hit
GGGCCTTCCCGGCCGTCCCGGAGCCGGTCGCGGCGCACCGCCACGGTGGA	260086	0.3851490725611636	No Hit
ACGAATGGTTTAGCGCCAGGTTCCACACGAACGTGCGTTCAACGTGACGG	247602	0.3666621066273818	No Hit
CGGCTTCGTCGGGAGACGCGTGACCGACGGTCCCCCCGGGACCCGACGGC	170383	0.25231213687083787   No Hit
...
```

In [ ]:
# Imports
import pandas as pd
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

Here is an example output: 

| / | Title |                                             Score | E-value |                
|------:|--------------------------------------------------:|--------:|-------------:|
    |     0 | Staphylococcus aureus...                      |    100.0 | 1.510770e-15 |  
|     1 | ... |    ... | ... |   


In [ ]:
# amount of overrepresented sequences it should blast (exercise = 5 but that takes a while)
nr = 2

# The output of the fastQC process has already been filtered a bit
df_overrepr = pd.read_table('data/overrepresented_sequences.txt', sep='\t') 
# Create df that will contain results
df_results = pd.DataFrame(columns=['Title','Score','E-value'])



# Only select 5 most overrepresented sequences
for i in range(nr-1):
    # Extract sequence
    seq = df_overrepr['#Sequence'][i]
    print(f'Sequence: {seq}, at index: {i}')
    # Make Seq for Biopython
    seq_blast = Seq(seq)
    # blast sequence
    print("Blasting")
    result_handle = NCBIWWW.qblast("blastn", "nt", seq_blast)

    # Parse the blast handle and extract only first record (without writing it to a file first)
    blast_record = NCBIXML.read(result_handle)
    
    # Each record has 
    descr = blast_record.descriptions[0]
    title = descr.title
    score = descr.score
    e = descr.e
    df_results = df_results.append({'Title': title, 
                   'Score': score,
                   'E-value': e}, ignore_index=True)
    print(df_results)
        
df_results

# 9. Two more exercises
The following two exercises are a bit longer and require a combination of the materials that we learned today (9.1) or dive into the world of proteins (9.2). The choice is yours as to which one might be more relevant.

## 9.1 Diagnosing Sickle Cell Anemia
[This link](https://krother.gitbooks.io/biopython-tutorial/content/sicklecell.html) will bring you to a great example exercise from Kristian Rother that combines all of the things that we learned today. 

Your goal is to develop an experimental test that reveals whether a patient suffers from the hereditary disease sickle cell anemia. The test for diagnosis should use a restriction enzyme on a patients’ DNA sample. For the test to work, you need to know exactly what genetic difference to test against. In this tutorial, you will use Biopython to find out.

The idea is to compare DNA and protein sequences of sickle cell and healthy globin, and to try out different restriction enzymes on them.

This tutorial consists of four parts:

1. Use the module Bio.Entrez to retrieve DNA and protein sequences from NCBI databases.
2. Use the module Bio.SeqIO to read, write, and filter information in sequence files.
3. Use the modules Bio.Seq and Bio.SeqRecord to extract exons, transcribe and translate them to protein sequences.
4. Use the module re to identify restriction sites. Regular expressions are not part of the course.

In [ ]:
# Check solutions with/at tuur.muyldermans@vib.be

## 9.2 Protein plots
Make two 3D plots of protein structures using the matplotlib pyplot library. For this you can use a Biopython module to retrieve the protein's PDB data and another one to parse it. 
1. The first one of the [human oxyhaemoglobin](https://www.rcsb.org/structure/1hho) chain A.
2. The second one with the superposition of chain B on top of chain A. 

In [ ]:
# Plot 1
from Bio.PDB import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

pdbl = PDBList()
pdbl.retrieve_pdb_file("1HHO")

parser = MMCIFParser()
struct = parser.get_structure("1HHO", "hh/1hho.cif")

# Part 1: Plot a structure
calphas = [res["CA"].get_coord() for res in struct[0]["A"] if "CA" in res]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot([xyz[0] for xyz in calphas],
        [xyz[1] for xyz in calphas],
        [xyz[2] for xyz in calphas], color='skyblue')
ax.view_init(30, 185)
plt.show()

In [ ]:
# Part 2: Plot an aligned pair of structures
chain_A = [res["CA"] for res in struct[0]["A"] if "CA" in res]
chain_B = [res["CA"] for res in struct[0]["B"] if "CA" in res][:141] # Explain superposition

sup = Superimposer()
sup.set_atoms(chain_A, chain_B)
sup.apply(chain_B)

calphas_A = [atom.get_coord() for atom in chain_A]
calphas_B = [atom.get_coord() for atom in chain_B]


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot([xyz[0] for xyz in calphas_A],
        [xyz[1] for xyz in calphas_A],
        [xyz[2] for xyz in calphas_A], color='green')
ax.plot([xyz[0] for xyz in calphas_B],
        [xyz[1] for xyz in calphas_B],
        [xyz[2] for xyz in calphas_B], color='red')
ax.view_init(30, 185)
plt.show()